In [2]:

# 导入futu-api
import futu as ft

# 实例化行情上下文对象
quote_ctx = ft.OpenQuoteContext(host="127.0.0.1", port=11111)

# 上下文控制
quote_ctx.start()              # 开启异步数据接收
quote_ctx.set_handler(ft.TickerHandlerBase())  # 设置用于异步处理数据的回调对象(可派生支持自定义)

# 低频数据接口
market = ft.Market.HK
code = 'HK.00123'
code_list = [code]
plate = 'HK.BK1107'

# Check available methods
print(dir(quote_ctx))  # List available methods


# print(quote_ctx.get_trading_days(market, start=None, end=None))   # 获取交易日
# print(quote_ctx.get_stock_basicinfo(market, stock_type=ft.SecurityType.STOCK))   # 获取股票信息
# print(quote_ctx.get_autype_list(code_list))                                  # 获取复权因子
# print(quote_ctx.get_market_snapshot(code_list))                              # 获取市场快照
# print(quote_ctx.get_plate_list(market, ft.Plate.ALL))                         # 获取板块集合下的子板块列表
# print(quote_ctx.get_plate_stock(plate))                         # 获取板块下的股票列表

# 高频数据接口
quote_ctx.subscribe(code, [ft.SubType.QUOTE, ft.SubType.TICKER, ft.SubType.K_DAY, ft.SubType.ORDER_BOOK, ft.SubType.RT_DATA, ft.SubType.BROKER])
print(quote_ctx.get_stock_quote(code))  # 获取报价
print(quote_ctx.get_rt_ticker(code))   # 获取逐笔
print(quote_ctx.get_cur_kline(code, num=100, ktype=ft.KLType.K_DAY))   #获取当前K线
print(quote_ctx.get_order_book(code))       # 获取摆盘
print(quote_ctx.get_rt_data(code))          # 获取分时数据
print(quote_ctx.get_broker_queue(code))     # 获取经纪队列

# 停止异步数据接收
quote_ctx.stop()

# 关闭对象
quote_ctx.close()

# 实例化港股交易上下文对象
trade_hk_ctx = ft.OpenHKTradeContext(host="127.0.0.1", port=11111)

# 交易接口列表
print(trade_hk_ctx.unlock_trade(password='123456'))                # 解锁接口
print(trade_hk_ctx.accinfo_query(trd_env=ft.TrdEnv.SIMULATE))      # 查询账户信息
print(trade_hk_ctx.place_order(price=1.1, qty=2000, code=code, trd_side=ft.TrdSide.BUY, order_type=ft.OrderType.NORMAL, trd_env=ft.TrdEnv.SIMULATE))  # 下单接口
print(trade_hk_ctx.order_list_query(trd_env=ft.TrdEnv.SIMULATE))      # 查询订单列表
print(trade_hk_ctx.position_list_query(trd_env=ft.TrdEnv.SIMULATE))    # 查询持仓列表

trade_hk_ctx.close()


2024-12-05 14:45:36,099 | 19332 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=30576892
['_OpenContextBase__host', '_OpenContextBase__port', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_callback_executor', '_check_subscribe_param', '_close', '_conn_id', '_connect_err', '_connect_sync', '_ctx_subscribe', '_get_sync_query_processor', '_handle_init_connect', '_handle_init_connect_rsp', '_handle_keep_alive', '_handler_ctx', '_init_connect_sync', '_is_encrypt', '_is_proc_run', '_keep_alive_fail_count', '_keep_alive_interval', '_last_keep_alive_time', '_last_recv_time', '_lock

In [3]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data, page_req_key = quote_ctx.request_history_kline('HK.HSImain', start='2019-09-11', end='2019-09-18', max_count=5)  # 每页5个，请求第一页
if ret == RET_OK:
    print(data)
    print(data['code'][0])    # 取第一条的股票代码
    print(data['close'].values.tolist())   # 第一页收盘价转为 list
else:
    print('error:', data)
while page_req_key != None:  # 请求后面的所有结果
    print('*************************************')
    ret, data, page_req_key = quote_ctx.request_history_kline('HK.HSImain', start='2019-09-11', end='2019-09-18', max_count=5, page_req_key=page_req_key) # 请求翻页后的数据
    if ret == RET_OK:
        print(data)
    else:
        print('error:', data)
print('All pages are finished!')
quote_ctx.close() # 结束后记得关闭当条连接，防止连接条数用尽

2024-12-05 15:01:32,436 | 23732 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=3, host=127.0.0.1, port=11111, user_id=30576892


         code        name             time_key     open    close     high  \
0  HK.HSImain  恒指主连(2412)  2019-09-11 00:00:00  26641.0  27109.0  27158.0   
1  HK.HSImain  恒指主连(2412)  2019-09-12 00:00:00  27080.0  27076.0  27335.0   
2  HK.HSImain  恒指主连(2412)  2019-09-13 00:00:00  27074.0  27309.0  27387.0   
3  HK.HSImain  恒指主连(2412)  2019-09-16 00:00:00  27307.0  27054.0  27394.0   
4  HK.HSImain  恒指主连(2412)  2019-09-17 00:00:00  27050.0  26747.0  27092.0   

       low  pe_ratio  turnover_rate  volume      turnover  change_rate  \
0  26641.0       0.0            0.0  213123  5.681636e+09     1.806369   
1  26940.0       0.0            0.0  165628  4.364643e+09    -0.121731   
2  26980.0       0.0            0.0  129083  3.423720e+09     0.860541   
3  26959.0       0.0            0.0  140046  3.759109e+09    -0.933758   
4  26676.0       0.0            0.0  150789  4.021585e+09    -1.134768   

   last_close  
0     26628.0  
1     27109.0  
2     27076.0  
3     27309.0  
4     27054.

1.3  获取历史K 线

In [13]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data, page_req_key = quote_ctx.request_history_kline('HK.HSImain', start='2019-09-11', end='2019-09-19',ktype=KLType.K_1M, max_count=100)  # 每页5个，请求第一页
if ret == RET_OK:
    print(data)
    print(data['code'][0])    # 取第一条的股票代码
    print(data['close'].values.tolist())   # 第一页收盘价转为 list
else:
    print('error:', data)
while page_req_key != None:  # 请求后面的所有结果
    print('*************************************')
    ret, data, page_req_key = quote_ctx.request_history_kline('HK.HSImain', start='2019-09-11', end='2019-09-12', max_count=1000, page_req_key=page_req_key) # 请求翻页后的数据
    if ret == RET_OK:
        print(data)
    else:
        print('error:', data)
print('All pages are finished!')
quote_ctx.close() # 结束后记得关闭当条连接，防止连接条数用尽


2024-12-05 15:35:41,787 | 23732 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=13, host=127.0.0.1, port=11111, user_id=30576892
          code        name             time_key     open    close     high  \
0   HK.HSImain  恒指主连(2412)  2019-09-11 00:00:00  26820.0  26832.0  26833.0   
1   HK.HSImain  恒指主连(2412)  2019-09-11 00:01:00  26833.0  26825.0  26836.0   
2   HK.HSImain  恒指主连(2412)  2019-09-11 00:02:00  26825.0  26815.0  26825.0   
3   HK.HSImain  恒指主连(2412)  2019-09-11 00:03:00  26814.0  26813.0  26815.0   
4   HK.HSImain  恒指主连(2412)  2019-09-11 00:04:00  26813.0  26800.0  26814.0   
..         ...         ...                  ...      ...      ...      ...   
95  HK.HSImain  恒指主连(2412)  2019-09-11 01:35:00  26809.0  26807.0  26810.0   
96  HK.HSImain  恒指主连(2412)  2019-09-11 01:36:00  26810.0  26813.0  26813.0   
97  HK.HSImain  恒指主连(2412)  2019-09-11 01:37:00  26814.0  26813.0  26816.0   
98  HK.HSImain  恒指主连(2412)  2019-09-11 01:38:00  26813.0  268